# Branch and Bound

Bei der systematischen Suche nach einer Lösung in einer sehr großen Menge von Kandidaten ist eine vollständige Breiten- oder Tiefensuche praktisch meist nicht mehr möglich. Man stelle sich z.B. ein 0/1 Rucksackproblem mit 50 Gegenständen vor. Bei diesem gibt es über eine Billiarde potentielle Belegungen, die nicht alle geprüft werden können. 

<img src="img/BnB_FullSearchTree.png" width="500">

Wenn eine vollständige Traversierung des Suchbaums nicht möglich ist, ist eine andere Herangehensweise gefragt.

Wie von der Tiefensuche bekannt ist, kann die Menge der Kandidaten (alle Blätter im Suchbaum, repräsentiert durch die Wurzel) systematisch in kleinere Teilmengen (Blätter unterhalb eines Knotenzweigs) unterteilt werden. Durch dieses **Verzweigen** (branch) entstehen Teilmengen, die jeweils unterschiedliche spezielle Eigenschaften besitzen. Die Menge der Kandidaten bei o.g. Rucksackproblem kann z.B. in die Teilmenge A (alle Rucksackbelegungen, bei denen Gegenstand 1 enthalten ist) und die Teilmenge B (alle Rucksackbelegungen, bei denen Gegenstand 1 nicht enthalten ist) verzweigt werden. Diese Mengen können nun wieder verzweigt bzw. in noch kleinere Teilmengen unterteilt werden bezüglich des Enthaltens der anderen Gegenstände. So entsteht ein typischer vollständiger Suchbaum.

Anstatt den Suchbaum vollständig zu durchlaufen, kann man sich die Frage stellen, welche Knoten es überhaupt wert sind, weiter  verfolgt zu werden. Möglicherweise ist die Wahrscheinlichkeit die Lösung unterhalb eines bestimmten Knotens zu finden größer als bei einem anderen. Vielleicht ist es sogar unmöglich, dass sich die Lösung unterhalb eines bestimmten Knotens befindet. In diesem Fall kann durch **Begrenzen** (bound) eine Expandierung solcher Knoten unterbunden werden. Um eine derartige Entscheidung treffen zu können, ist es notwendig, Informationen, die an einem Knoten bereits vorliegen oder einfach ermittelt werden können, genau zu analysieren. 

Die generelle Vorgehensweise beim Entwickeln eines **Branch and Bound Algorithmus'** beinhaltet zunächst den **Entwurf des Suchbaums**. Da am Ende nur die Blätter (die möglichen Lösungen) interessant sind, kann der Baum oberhalb dieser Blätter beliebig gestaltet werden. Des Weiteren wird eine **Bewertungsfunktion** benötigt. Diese muss für einen Knoten entscheiden, ob dieser expandiert werden soll oder vernachlässigt werden kann. Zusätzlich sollte eine **Reihenfolge** festgelegt werden in der die Knoten verarbeitet werden sollen. Zur effektiven Begrenzung ist es notwendig, möglichst schnell zu guten Teillösungen zu gelangen, um dadurch enge Schranken für die Bewertungsfunktion zu erhalten.

Sind diese drei Eigenschaften optimal aufeinander abgestimmt, kann die Suche nach einer Lösung sehr viel effizienter werden. Dies ist aber nicht zwangsläufig der Fall. Können z.B. keine Knoten ausgeschlossen werden, dann wird praktisch eine vollständige Suche durchgeführt, die durch die zusätzliche Bewertung am Ende langsamer ist als z.B. eine einfache Tiefensuche. Oder wenn die Bewertung ineffizient ist, könnte der dadurch entstehende Verlust den Gewinn durch die Begrenzung aufheben oder gar übersteigen.

## 0/1 Rucksackproblem

Für ein 0/1 Rucksackproblem mit 4 Gegenständen könnte genau der oben gezeigte Suchbaum entstehen. 

Auf jeder Ebene wird für einen bestimmten Gegenstand die Entscheidung getroffen, ob dieser im Rucksack enthalten sein soll oder nicht. Wurden 4 Entscheidungen getroffen, steht eine mögliche Rucksackbelegung fest. Diese kann nun die Lösung sein oder auch nicht.

### Statische Begrenzung
Um diesen Baum zu begrenzen, müssen Möglichkeiten gefunden werden, bereits für einen Knoten zu entscheiden ob die Lösung unter diesem zu finden ist. Beim Rucksackproblem bietet die Kapazität des Rucksacks eine solche.

<img src="img/BnB_TreeBoundStatic.png" width="500">

Für vier Gegenstände mit den Gewichten 5,4,3,2 und einer Maximalkapazität von 8 können die Zweige ignoriert werden, bei denen die Kapazitätsgrenze überschritten wird (grau). Von 16 möglichen Kandidaten bleiben noch 10 übrig und anstatt 15 Expandierungen vorzunehmen, sind nur noch 9 erforderlich.

### Dynamische Begrenzung
Die Kapazitätsgrenze ist eine statische Schranke, denn sie ändert sich während der Suche nicht. Besser sind immer **dynamische Schranken** die im Verlauf der Suche immer strenger werden. Wurde z.B. bereits irgendeine gültige Rucksackbelegung gefunden, können Knoten unter denen nur schlechtere Belegungen zu finden sind ignoriert werden. Je besser der aktuell beste Rucksack im Verlauf der Suche wird, umso strenger wird die Schranke und umso stärker fällt die Begrenzung aus. Das Ziel ist also möglichst schnell zu einer guten (noch nicht optimalen) Lösung zu gelangen, um von einer möglichst starken Begrenzung zu profitieren.

Gibt es z.B. bereits einen gültigen Rucksack mit einem gewissen Wert, so müsste für einen Knoten (quasi einen noch nicht fertig gefüllten Rucksack) bestimmt werden, ob überhaupt noch ein gültiger Rucksack zusammengestellt werden kann, der wertvoller ist. An jedem Knoten hat der untersuchte Teil-Rucksack einen momentanen Wert und eine Restkapazität. Um zu bestimmen, welcher zusätzliche Wert unter Verbrauch der Restkapazität noch hinzugefügt werden kann, müsste allerdings ein weiteres (kleineres) Rucksackproblem gelöst werden. **Die Effizienz einer Bewertungsfunktion ist aber entscheidend.** Diese Effizienz kann dadurch erreicht werden, dass eine weniger genaue Schranke ermittelt wird. Für das 0/1 Rucksackproblem bietet sich z.B. folgende Möglichkeit:

| Gegenstand $i$ | Gewicht $g$  | Wert $w$    | spezifischer Wert $w/g$ |
|:--------------:|:------------:|:-----------:|:-----------------------:|
| 1              | 5            | 9           | 1,8                     |
| 2              | 3            | 5           | 1,7                     |
| 3              | 4            | 6           | 1,5                     |
| 4              | 2            | 3           | 1,5                     |

Die $n$ Gegenstände mit den Werten $w_i$ und den Gewichten $g_i$ für $1 \leq i \leq n$ können nach ihrem *spezifischen Wert* $\frac{w_i}{g_i}$ sortiert entschieden werden. Dieser Quotient sagt aus, welcher Wert pro Gewichtseinheit beim Hinzufügen dieses Gegenstands hinzugewonnen wird. Entscheidungen für Gegenstände mit hohem Wert und geringem Gewicht werden so zuerst getroffen. Es ist sichergestellt, dass $\frac{w_i}{g_i} \geq \frac{w_{i+1}}{g_{i+1}}$. Für einen Knoten (einen Teil-Rucksack) bei dem bereits $k$ Gegenstände entschieden wurden, kann bei gegebener Maximalkapazität $K$ die Restkapazität $r$ und der damit einhergehende maximal erreichbare Rucksackwert $m$ bestimmt werden.
(Ein (Teil-)Rucksack ist definiert durch den Vektor $\overrightarrow{x}=(x_1, x_2, \dots, x_k) \text{ für } 1 \leq k \leq n \text{ und } x_i \in \{0,1\}$. Der Rucksack $(1,0,1)$ enthält demnach Gegenstand 1 und 3 und nicht Gegenstand 2.)
$$\\
r = K - \sum_{i=1}^k x_i g_i\\
m = r \frac{w_{k+1}}{g_{k+1}} + \sum_{i=1}^k x_i w_i
$$
Die Restkapazität $r$ eines Rucksacks ist die Differenz zwischen Maximalkapazität $K$ und aktuellem Rucksackgewicht. Der maximal erreichbare Rucksackwert $m$ ist die Summe aus dem aktuellem Rucksackwert und der Restkapazität multipliziert mit dem spezifischen Wert des **nächsten** noch nicht entschiedenen Gegenstands ($k+1$). Da die spezifischen Werte aller weiteren Gegenstände maximal genauso hoch sind, bildet $m$ eine obere Schranke. Liegt der Wert des momentan besten Rucksacks bereits über diesem maximal erreichbaren Wert, so kann der entsprechende Knoten ignoriert werden.

### Ablauf

<img src="img/BnB_TreeBoundRest.png" width="600">

Der mögliche Wertzugewinn kann so für jeden Knoten sehr einfach berechnet werden. Obiges Bild zeigt die entsprechenden Zugewinne an den Knoten für die rechts aufgelisteten Gegenstände. Wie effektiv die resultierende Begrenzung ist, hängt nun von der Reihenfolge ab in der die Knoten geprüft werden. In jedem Fall muss bei Knoten <font color='red'>A</font> begonnen werden. Bei <font color='red'>A</font> ist der aktuelle Rucksack leer (hat den Wert $0$) und es ist ein Zugewinn von maximal $8\cdot 1.8 = 14.4$ zu erwarten. Da $0+14.4$ größer ist als der momentan beste Rucksack (ist zu Beginn ebenfalls leer, d.h. Wert gleich Null) muss <font color='red'>A</font> expandiert werden. Durch die Expandierung wird ein Rucksack gefunden der Gegenstand 1 enthält und somit den Wert $9$ hat. Dieser wird nun zum momentan besten bekannten Rucksack und gibt mit seinem Wert eine erste Schranke vor. Als nächste Knoten kommen nun <font color='red'>B</font> und <font color='red'>C</font> in Frage. Bei <font color='red'>B</font> ist der aktuelle Rucksackwert $9$ und durch den Zugewinn von $5$ kann ein maximaler Rucksackwert von $14$ erreicht werden. Bei <font color='red'>C</font> ist der maximal zu erreichende Wert nur $13.3$. Aus diesem Grund sollte Knoten <font color='red'>B</font> bevorzugt behandelt werden. Folgende Tabelle zeigt die weitere Entwicklung:

| Schritt | Knoten | Rucksackwert | Zugewinn | Max | Aktion | Best |
| --- | --- | --- | --- | --- | --- | --- |
| 1 | A | 0 | 14,4 | 14,4 | Expandiere zu B und C | 9 |
| 2 | B | 9 | 5 | 14 | Expandiere zu D und E | 14 |
|   | C | 0 | 13,3 | 13,3 | Aufschieben |  |
| 3 | C | 0 | 13,3 | 13,3 | Bound (Max $\leq$ Best) |  |
|   | D | 14 | 0 | 14 | Gehe zu H | 14 |
|   | E | 9 | 4,5 | 13,5 | Bound (Max $\leq$ Best) |  |
| 4  | H | 14 | 0 | 14 | Ende (keine Knoten mehr) | 14 |

Die Expandierung von <font color='red'>B</font> in Schritt 2 führt zu einem neuen besten Rucksack wodurch die Bound-Schranke auf 14 angehoben wird. In Schritt 3 kann für die verfügbaren Knoten <font color='red'>C</font> und <font color='red'>E</font> festgestellt werden, dass sich unterhalb keine bessere Lösung befinden kann. Damit ergibt sich folgender dynamisch reduzierter Suchbaum bei dem weitere 6 Expandierung eingespart wurden.

<img src="img/BnB_TreeBoundResult.png" width="600">

### Implementation

Nur selten werden Branch and Bound Algorithmen rekursiv implementiert. Eine *sortierte Queue (priority queue)* ist zumeist die beste Option, um eine Menge abzuarbeitender Elemente zu verwalten, so wie es hier für die Knoten erforderlich ist. Die meisten höheren Programmiersprachen bieten hierzu entsprechende Datenstrukturen an. Hier wird *heapq* verwendet, um die zur Expandierung zur Verfügung stehenden Knoten nach ihrem maximal erreichbaren Rucksackwert sortiert bereitzustellen. Die Auswahl des als nächstes zu expandierenden Knotens kann so durch ein einfaches *heappop* realisiert werden.

Die Expandierung, also das Erzeugen neuer Knoten mit und ohne dem nächsten Gegenstand, erfolgt schlicht durch Anhängen von 1 oder 0 an die Liste welche einen Rucksack repräsentiert. Die expandierten Knoten werden nur dann der Queue hinzugefügt, wenn die Kapazitätsgrenze nicht überschritten wird und der maximal erreichbare Wert den momentan besten Rucksackwert überschreitet.

Gibt es keine zu prüfenden Knoten mehr, also wurden alle Knoten entweder bereits verarbeitet oder ausgeschlossen, steht das Endergebnis fest.

In [1]:
import heapq # Priority Queue
K = 8
allitems = [(5, 9), (3, 5), (4, 6), (2,3)]

def bnbKnapsack(items, K):
    # Sortierung der Gegenstände nach Gewicht pro Werteinheit (kleiner ist besser)
    sortedItems = sorted(items, key=lambda x: x[0] / x[1])
    bestKS = []  # [0,1,1,0] heißt das item1 und 2 enthalten sind und item0 und 3 nicht, anfangs leer
    # Hilfsfunktion zur Bestimmung des Rucksackwerts
    def ksValue(ks): return sum([sortedItems[i][1] if ks[i] == 1 else 0 for i in range(len(ks))])
    # Hilfsfunktion zur Bestimmung des Rucksackgewichts
    def ksWeight(ks): return sum([sortedItems[i][0] if ks[i] == 1 else 0 for i in range(len(ks))])
    # Hilfsfunktion zur Bestimmung des maximal erreichbaren Rucksackwerts
    def limit(ks): return ksValue(ks)+(K-ksWeight(ks))*sortedItems[len(ks)][1]/sortedItems[len(ks)][0]

    knapsacks = []  # die aktuellen Knoten, sortiert nach maximal erreichbarem Rucksackwert
    heapq.heappush(knapsacks, (1 / limit([]), []))  # Wurzel ist der leere Rucksack
    while len(knapsacks): # solange es Knoten zu prüfen gibt
        ks = heapq.heappop(knapsacks)[1]  # Knoten mit dem höchsten erreichbaren Rucksackwert als nächstes
        print("check:", ks)
        for x in [0,1]: # Expandieren zu 2 neuen Knoten
            expanded = ks + [x]
            print("expanded:", expanded, end=" ")
            if ksWeight(expanded) <= K:  # Kapazitätsgrenze noch nicht überschritten
                best = ksValue(bestKS) # momentan bester Rucksack
                if len(expanded) < len(sortedItems):  # noch kein vollständiger Rucksack
                    m = limit(expanded) # maximal zu erreichender Wert
                    if m > best:  # könnte ein besserer Rucksack erzielt werden?
                        heapq.heappush(knapsacks, (1 / m, expanded))
                        print("queued with max:", m)
                    else:
                        print("bound")
                if best < ksValue(expanded):  # besser?
                    bestKS = expanded
                    print(expanded, "better")
            else:
                print("too heavy")
        
    return bestKS  


print("Best:",bnbKnapsack(allitems, K))
        

check: []
expanded: [0] queued with max: 13.333333333333334
expanded: [1] queued with max: 14.0
[1] better
check: [1]
expanded: [1, 0] queued with max: 13.5
expanded: [1, 1] queued with max: 14.0
[1, 1] better
check: [1, 1]
expanded: [1, 1, 0] bound
expanded: [1, 1, 1] too heavy
check: [1, 0]
expanded: [1, 0, 0] bound
expanded: [1, 0, 1] too heavy
check: [0]
expanded: [0, 0] bound
expanded: [0, 1] bound
Best: [1, 1]


## Traveling Salesman Problem (TSP)

Das oft zitierte Problem des Handlungsreisenden besteht darin, eine gewisse Anzahl von Städten genau einmal zu besuchen und zum Ausgangspunkt zurückzukehren, wobei der zurückgelegte Gesamtweg minimal ist. Die Wegstrecken zwischen den Städten sind bekannt. 

Dieses Problem haben aber nicht nur Handlungsreisende, sondern auch Logistikunternehmen (wie muss das Postauto fahren, sodass die Tour möglichst schnell geht), Platinen-Designer (wie müssen Kontakte verbunden werden, sodass möglichst wenig Edelmetall verbraucht wird) und viele mehr. In jedem Fall geht es darum, irgendwelche "Kosten" zu minimieren. 

<img src="img/BnB_TSPGraph.png" width="300">

Ein Traveling Salesman Problem (TSP) lässt sich sehr einfach in Form von gewichteten Graphen darstellen. Die Gewichte der Kanten entsprechen hier den Kosten für den Übergang von einem Knoten zum anderen (z.B. dem Preis für eine Flugreise von Ort A nach Ort B). Die Kosten zwischen zwei Knoten können dabei auch von der Richtung abhängig sein.

Die Suche nach einem Weg mit minimalen Kosten kann auch hier mit einem Entscheidungsbaum visualisiert werden. Da es sich um eine Rundreise handelt, spielt es keine Rolle welcher Knoten den Beginn (die Wurzel des Baums) darstellt.

<img src="img/BnB_TSPTree.png" width="500">

Gibt es $n$ Knoten, so gibt es im ersten Schritt $n-1$ Möglichkeiten die Reise fortzusetzen, im nächsten Schritt nur noch $n-2$ usw. bis nach $n-2$ Schritten nur ein Knoten als mögliches Ziel verbleibt. Mit der Rückkehr zum Ausgangspunkt schließt sich der Kreis und die Länge des Weges steht fest.

### Begrenzung

Um einen Knoten ignorieren zu können, muss eine Bewertung durchgeführt werden, die entscheidet ob die Expandierung des Knotens zielführend ist oder nicht. Dafür muss neben dem bekannten bis zum Knoten bereits zurückgelegten Weg auch die minimale Länge des Folgeweges berechnet werden. Letzeres exakt zu bestimmen führt zu einem verkleinerten TSP dessen Lösung nur in exponentieller Zeit erfolgen kann. Diese Ineffizienz ist für eine Bewertungsfunktion nicht akzeptabel. Es muss also eine Approximation durchgeführt werden, die weniger aufwendig aber dennoch ausreichend ist.

Als Datenstruktur zur Abbildung eines TSP eignet sich eine Entfernungsmatrix. Hier kann zu jeder Kombination zweier Knoten der entsprechende Kantenwert hinterlegt werden. Existiert eine Kante nicht, so können die Kosten für den entsprechenden (nicht existenten) Knotenübergang als unendlich groß angenommen werden.

|  | 0 | 1 | 2 | 3 | **Min** |
| --- | --- | --- | --- | --- | --- |
| **0** | $\infty$ | $20_{10}$ | $15_5$ | $10_0$ | **$10$** |
| **1** | $8_0$ | $\infty$ | $9_1$ | $8_0$ | **$8$** |
| **2** | $6_0$ | $12_6$ | $\infty$ | $13_7$ | **$6$** |
| **3** | $5_0$ | $10_5$ | $9_4$ | $\infty$ | **$5$** |
| **Min** | **$0$** | **$5$** | **$1$** | **$0$** | **$\mathbf{35}$** |

Eine solche Entfernungsmatrix kann wie in obiger Tabelle gezeigt reduziert werden. Zunächst muss von jeder Zeile das Minimum ermittelt werden (letzte Spalte). Anschließend wird das Zeilen-Minimum von jedem Wert der Zeile abgezogen (Fussnoten, beachten: $\infty - x = \infty\;|\;x \neq \infty$). Schlussendlich werden spaltenweise die Minima dieser Differenzen berechnet und alle Zeilen- und Spalten-Minima addiert. 

**Hintergrund:** *Ein Zeilen-Minimum steht für die geringste Entfernung zwischen dem Zeilen-Knoten und irgendeinem anderen Knoten. Stehen alle Zeilen-Minima fest, ist deren Summe der theoretisch kürzestmögliche Weg, um an allen Knoten genau ein Mal vorbei zu kommen. Praktisch kann es sein, dass diese rein theoretische Weglänge nicht wirklich erreicht werden kann. Der Teilweg 0-3-0 im Beispiel oben (entlang der minimalen Kosten) ist z.B. kein gültiger Weg. Bei dieser Vorgehensweise wird somit in Kauf genommen, dass die Kanten mit den geringsten Kosten möglicherweise gar nicht in ein und demselben Weg vorkommen können. Dieser Fehler wird zu einem gewissen Grad kompensiert, indem nach dem Subtrahieren der Zeilen-Minima die Spalten-Minima der Differenzen gebildet werden. Angenommen die Kanten mit den minimalen Kosten würden einen gültigen Weg bilden, dann gäbe es nach Subtraktion der Minima in jeder Spalte eine 0 wodurch die Summe der Spalten-Minima 0 würde. Ist dem nicht so, kann die Summe der Spalten-Minima als minimale Größe des Fehlers der Approximation angesehen werden und zur Summe der Zeilen-Minima addiert werden. Der kürzestmögliche Weg unterhalb des betrachteten Knotens kann somit niemals kürzer werden als die berechnete Summe.*

### Ablauf

Beim Durchlaufen des o.a. Entscheidungsbaum werden mit jeder Expandierung alle möglichen weiteren Knoten des Weges beschritten. Mit der Festlegung eines solchen Schritts ändert sich die Entfernungsmatrix, denn mit jedem Schritt fallen all die Kanten weg, die im weiteren Verlauf nicht mehr genutzt werden dürfen. Um eine Kante "unbegehbar" zu machen, muss nur der entsprechende Wert in der Entfernungsmatrix auf unendlich gesetzt werden. Ein Schritt von Knoten $i$ zu Knoten $j$ macht jede andere Kante hin zum Knoten $j$ und weg vom Knoten $i$ unbegehbar. Auch die Gegenrichtung von $j$ nach $i$ darf nicht mehr beschritten werden. In folgender Tabelle sind die Modifikationen für den Übergang von Knoten 0 zu Knoten 3 rot hervorgehoben. Auch hier kann wieder die Mindestlänge approximiert werden.

|0 >> 3| 0 | 1 | 2 | 3 | **Min** |
| --- | --- | --- | --- | --- | --- |
| **0** | $\infty$ | $\color{red}\infty$ | $\color{red}\infty$ | $10_0$ | **$10$** |
| **1** | $8_0$ | $\infty$ | $9_1$ | $\color{red}\infty$ | **$8$** |
| **2** | $6_0$ | $12_6$ | $\infty$ | $\color{red}\infty$ | **$6$** |
| **3** | $\color{red}\infty$ | $10_{\color{red}1}$ | $9_{\color{red}0}$ | $\infty$ | **$\color{red}9$** |
| **Min** | **$0$** | **${\color{red}1}$** | **${\color{red}0}$** | **$0$** | **$\mathbf{{\color{red}{34}}}$** |

Bei der Expandierung eines Knotens wird für jeden entstehenden Knoten eine solche modifizierte Matrix mit einer entsprechenden approximierten Mindestlänge berechnet. Im weiteren Verlauf kann nun mit dem erfolgversprechendsten Knoten (mit der kleinsten Mindestlänge) die Expandierung fortgesetzt werden. Im besten Fall kann das Ergebnis so direkt angesteuert werden.

<img src="img/BnB_TSPBound.png" width="500">

So optimal wie in diesem Beispiel verläuft die Suche jedoch nicht in jedem Fall. Die nach der ersten Expandierung getroffene Entscheidung zur Weiterverfolgung eines Knotens kann durchaus zu Knoten führen, die eine größere Mindestlänge aufweisen als die bereits in einem vorherigen Schritt erzeugten Knoten. In einer solchen Situation sollten dann, streng nach der Regel **best-first**, eben diese vorher vernachlässigten Knoten expandiert werden.

### Implementation
Die Entfernungsmatrix kann sehr einfach als zweidimensionales Array erstellt werden. Zur Repräsentation von Unendlich bieten die meisten höheren Programmiersprachen eine entsprechende Möglichkeit. Mit den Hilfsfunktionen delta und modify erfolgt die Berechnung der Schranke und die Anpassung der Entfernungsmatrix bei Festlegung eines Übergangs. Der eigentliche Algorithmus basiert auf einer Priority Queue die stets den erfolgversprechendsten Knoten zur Expandierung bereitstellt (mit minimaler theoretisch erreichbarer Mindestlänge). Sobald ein Endpunkt erreicht ist (alle Knoten sind im Weg enthalten) ist die kürzeste Rundreise gefunden. Anderenfalls erfolgt die Expansion durch Modifizierung der Matrix und Berechnung der zugehörigen Schranke. Die Begrenzung erfolgt in diesem Fall indirekt durch Verzögerung der Expansion von Knoten mit einer hohen Schranke. 

In [2]:
inf = float("inf")
distanceMatrix = [
    [inf, 20, 15, 10],
    [8, inf, 9, 8],
    [6, 12, inf, 13],
    [5, 10, 9, inf]
] # kürzeste Rundreise: 35

def delta(d):
    rd = [] # reduzierte Matrix
    s = len(d)
    delta = 0
    for row in range(s):
        rowmin = min(d[row]) # Zeilen-Minimum
        rd.append([d[row][i] - rowmin for i in range(s)]) # Zeile mit Differenzen
        delta += rowmin # Minima summieren
    for col in range(s):
        colmin = min([rd[i][col] for i in range(s)]) # Spalten-Minimum der Differenzen
        delta += colmin # Fehler addieren
    return delta

def modify(d, fromP, toP):
    m = [] # modifizierte Matrix
    s = len(d)
    for row in range(s): m.append(d[row].copy()) # Kopie
    for row in range(s): # Wege zu toP = unendlich
        if row != fromP: m[row][toP] = inf
    for col in range(s): # Wege von fromP unendlich
        if col != toP: m[fromP][col] = inf
    m[toP][fromP] = inf # Weg zurück unendlich
    return m

import heapq
def bnb_TSP(d):
    s = len(d)
    routes = []
    heapq.heappush(routes, (0, ([0], d)))
    while len(routes):
        node = heapq.heappop(routes) # Knoten mit dem niedrigsten delta
        item = node[1]
        print("check:",item[0], node[0])
        if len(item[0]) == s: # kürzester Weg ist vollständig
            return (item[0]+[item[0][0]], node[0])
        last = item[0][-1] # zuletzt besuchter Knoten
        matrix = item[1] # Ausgangsmatrix
        for x in range(s):
            # für jeden noch nicht besuchten Knoten der nicht unendlich weit weg ist
            if x in item[0] or matrix[last][x] == inf: continue
            m = modify(matrix, last, x) # Matrix modifizieren
            dt = delta(m) # theoretisch erreichbare Mindestweglänge
            print("  extract:", item[0]+[x], dt)
            heapq.heappush(routes, (dt, (item[0]+[x], m))) # in Priority Queue einordnen

print("best:", bnb_TSP(distanceMatrix));

check: [0] 0
  extract: [0, 1] 40
  extract: [0, 2] 40
  extract: [0, 3] 34
check: [0, 3] 34
  extract: [0, 3, 1] 35
  extract: [0, 3, 2] 39
check: [0, 3, 1] 35
  extract: [0, 3, 1, 2] 35
check: [0, 3, 1, 2] 35
best: ([0, 3, 1, 2, 0], 35)
